# Predicting car prices with a randomforest or XGBoost

Use a machine learning model from snowpark ML to predict the price of a car given some of its features. 

* KM stand
* age
* Model / Make
* Transmission

The data is scraped from gaspedaal.nl in 2023

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark import functions as F

import plotly.express as px

from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.ensemble import RandomForestRegressor

from snowflake.ml.modeling.metrics import mean_absolute_percentage_error
from snowflake.ml.modeling.metrics import r2_score

from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import OneHotEncoder

from snowflake.ml.registry import Registry

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Get cars data set

The data table is called CAR_PRICES and is in the snowflake CARS_DATA database

In [ ]:
#### use snowpark to get to the data
cars_sf = session.table('CARS_DATA.PUBLIC.CAR_PRICES')

### Add two columns
cars_sf = (
    cars_sf
    .with_column('age' , 2023 - cars_sf['BOUWJAAR'])
    .with_column('N_doors', cars_sf["DEUR"].substring(1,2))
)

In [ ]:
#### cleanup data by removing some outliers
cars_clean = (
    cars_sf
    .filter(F.col("KM_STAND") <= 500000)
    .filter(F.col("AGE") <= 20 )
    .filter(F.col("TRANSMISSIE").in_(F.lit("Handgeschakeld"), F.lit("Automaat")) )
    .filter(F.col("VRAAGPRIJS") <= 100000)
    .filter(F.col("BRANDSTOF").in_(F.lit("Benzine"), F.lit("Diesel")) )
)

## drop the columns that we don't need
cars_clean = cars_clean.drop(["PRIJS","VERMOGEN"] )

In [ ]:
cars_clean.describe()

### Filter on car brands that have at least 750 records

In [ ]:
top_merken = (
    cars_clean
    .group_by("MERK")
    .agg(F.count("MERK").alias('N'))
    .filter(F.col('N') > 750)
    .withColumnRenamed('MERK', 'MERK2')
)
top_merken

In [ ]:
cars_clean_topmerken = (
    cars_clean
    .join(top_merken, cars_clean["MERK"] == top_merken["MERK2"], "inner")
    .select(cars_clean["*"])
)

In [ ]:
cars_clean_topmerken.count()

In [ ]:
cars_clean_topmerken

## Predictive Modeling


In [ ]:
### Define the features in the model
CATEGORICAL_COLUMNS = ["MERK", "BRANDSTOF", "TRANSMISSIE", "KLEUR"]

### names of the onehot encoded
CATEGORICAL_COLUMNS_OHE = ["MERK_OE" ,"BRANDSTOF_OE", "TRANSMISSIE_OE", "KLEUR_OE"]

NUMERICAL_COLUMNS = [ 'AGE', 'KM_STAND']

### Define the target
LABEL_COLUMNS = ['VRAAGPRIJS']
OUTPUT_COLUMNS = ['PREDICTED_PRICE']

In [ ]:
# Define the random forest regression model
regressor_model = RandomForestRegressor(
    n_estimators = 250, 
    max_depth = 6,
    n_jobs = -1,
    min_samples_split = 500,
    min_samples_leaf = 500,
    label_cols = LABEL_COLUMNS,
    output_cols= OUTPUT_COLUMNS
)

In [ ]:
### only use the columns we need

cars_clean_model_data = (
    cars_clean_topmerken
    .select(CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS + LABEL_COLUMNS)
)

cars_clean_model_data

In [ ]:
### split into train and test
df_train, df_test = cars_clean_model_data.random_split(weights=[0.80, 0.20], seed=0) 

### Machine learning Pipeline  

Categorical columns need to be one-hot encoded first before we can use them in a random forest model

We can build a pipeline to doe the onehot encoding and then the model training


In [ ]:
# ML pipeline
ml_pipe = Pipeline(
    steps = [
        (
            "ohe", 
            OneHotEncoder(
                input_cols=CATEGORICAL_COLUMNS, 
                output_cols = CATEGORICAL_COLUMNS_OHE, 
                drop_input_cols = True
            )
        ),
        (
            "regressionmodel",
            regressor_model
        )
    ]
)

In [ ]:
ml_pipe

### fit the machine learning pipeline

In [ ]:
### Train the pipeline using fit_predict, 

# so applying first onehot encoding
# then train the randomforest model
# apply the model so that the train data has an extra column of predictions
# output of fit_predict is a snowpark data frame

df_train_out = ml_pipe.fit_predict(df_train)


In [ ]:
df_train_out

In [ ]:
### We can now apply the fitted model on the test set
df_test_processed = ml_pipe.predict(df_test)
df_test_processed

### Model metrics
On the test set we can calculate certain model metrics

In [ ]:
mape = mean_absolute_percentage_error(
    df=df_test_processed, 
    y_true_col_names="VRAAGPRIJS", 
    y_pred_col_names="PREDICTED_PRICE"
)
mape

In [ ]:
r2 = r2_score(
    df=df_test_processed, 
    y_true_col_name="VRAAGPRIJS", 
    y_pred_col_name="PREDICTED_PRICE"
)
r2

### Feature importance

The regression model itself is the last step of the ml pipeline, we can extract it to get for example the feature importances. But the snowpark ML randomforest model is merely a wrapper to the scikit learn object. We need to get to that so that we can extract feature importances

In [ ]:
sk_model = ml_pipe.steps[-1][1].to_sklearn()

In [ ]:
feature_importances_df = pd.DataFrame({
    'Feature': sk_model.feature_names_in_,
    'Importance': sk_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [ ]:
feature_importances_df

In [ ]:
### plot prediction vs 
### Note that we bring over data to pandas
test_df = df_test_processed.to_pandas()

In [ ]:
fig = px.scatter(
    test_df,
    x="VRAAGPRIJS", y="PREDICTED_PRICE"
)
fig

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(test_df, x="VRAAGPRIJS", y="PREDICTED_PRICE")

# Streamlit app

In [ ]:
merken_lijst = top_merken.to_pandas().MERK2.values

In [ ]:
c1,c2,c3 = st.columns(3)
with c2:
    AGE = st.number_input("Age",0,20,2)
    BRANDSTOF = st.selectbox("BRANDSTOF", [ "Benzine", "Diesel"])
with c3:
    KM_STAND = st.number_input("KM_STAND",15000)
    TRANSMISSIE = st.selectbox("TRANSMISSIE",["Handgeschakeld", "Automaat"])
with c1:
    MERK = st.selectbox("MERK", merken_lijst)

st.divider()
### convert input to a pandas data frame so that we can convert it to snowpark dataframe

new_car = pd.DataFrame({
    "MERK": [MERK],
    "AGE" : [AGE],
    "KM_STAND": [KM_STAND],
    "TRANSMISSIE" :[TRANSMISSIE],
    "BRANDSTOF" : [BRANDSTOF]
    
})

new_car_sf = session.create_dataframe(new_car)

### predict price of the new car
ml_pipe.predict(new_car_sf).select("PREDICTED_PRICE")

# Model registry

In [ ]:
-- create a separate schema for the model registry
USE DATABASE CARS_DATA;
CREATE SCHEMA registry;
SHOW SCHEMAS;

In [ ]:
## create and establish a link to the registry 

from snowflake.ml.registry import Registry

reg = Registry(session=session, database_name="CARS_DATA", schema_name="REGISTRY")

reg

In [ ]:
mv = reg.log_model(
    ml_pipe,
    model_name="car_prediction_random_Forrest",
    version_name="v1",
    comment="My awesome ML model"
)

In [ ]:
mv

In [ ]:
reg.show_models()

In [ ]:
m = reg.get_model("CAR_PREDICTION_RANDOM_FORREST")
m

In [ ]:
m.versions()

In [ ]:
mv.set_metric("test_R2", 0.70)

### use model from registry to predict on new data

In [ ]:
new_prediction = mv.run(cars_clean_model_data, function_name="predict")
new_prediction.show()

### SQL interface to models

registered models in the model registry can also be used to score new data with SQL!!!


In [ ]:
SHOW MODELS

There is a PREDICT METHOD THAT WE CAN USE in SQL

In [ ]:
SHOW FUNCTIONS IN MODEL CAR_PREDICTION_RANDOM_FORREST

We need some cars data that we can use for scoring. Lets just use snowpark data frame we already have....

In [ ]:
### save data frame as table for scoring
cars_clean_model_data.write.mode("overwrite").save_as_table("CARS_DATA.PUBLIC.CARS_TO_SCORE")

In [ ]:
SELECT * FROM CARS_DATA.PUBLIC.CARS_TO_SCORE

We used KLEUR, but there were some NULL values, the model cant handle that 

In [ ]:
SELECT * FROM CARS_DATA.PUBLIC.CARS_TO_SCORE WHERE KLEUR IS NULL

In [ ]:
SELECT 
    * ,
    CAR_PREDICTION_RANDOM_FORREST!PREDICT(
        CARS_TO_SCORE.MERK,
        CARS_TO_SCORE.BRANDSTOF,
        CARS_TO_SCORE.TRANSMISSIE,
        CARS_TO_SCORE.KLEUR,
        CARS_TO_SCORE.AGE,
        CARS_TO_SCORE.KM_STAND
    )
    as prediction_dict ,
    prediction_dict['PREDICTED_PRICE']::double as PREDICTED_PRICE,
  
FROM 
    CARS_TO_SCORE
 WHERE 
    TRANSMISSIE = 'Handgeschakeld' AND 
    KLEUR IS NOT  NULL
;